#학번 숫자 인식 및 분류

#분류 모델

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
%matplotlib inline 

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
#랜덤값 고정
tf.random.set_seed(1234)

#normalize
X_train = X_train / 255.0
X_test = X_test / 255.0

# reshape
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# one-hot 인코딩
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
#모델 컴파일
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, input_shape=(28,28,1), padding='same', activation='relu'), 
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, input_shape=(28,28,1), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, input_shape=(28,28,1), padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])
cnn_model.compile(loss='categorical_crossentropy', optimizer= tf.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy'])

In [ ]:
#callback & early stopping 지점, 기준
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best-model.h5', save_best_only = True)
earlystopping_cb = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights=True)

#모델 학습 + callback & early stopping
history = cnn_model.fit(X_train, y_train, batch_size=100, epochs=20, validation_data=(X_test, y_test), callbacks = [checkpoint_cb, earlystopping_cb])

In [ ]:
#callback & early stopping한 지점까지 에포크 단위별 loss 시각화
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
#모델 정확도 / 그래프 해석 예시: 그래프에서 보이듯이 epoch 5에서 가장 낮은 손실 점수 보여준다. (인덱스0부터 시작이므로 4는 5를 의미)
result = cnn_model.evaluate(X_test, y_test)
print("예측 정확도(%): ", result[1]*100)

#데이터베이스에 넣을 feature_vector값 구하기 (등록)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

# resnet적용
model = models.resnet18(pretrained=True)

# avgpool layer 추출
layer = model._modules.get('avgpool')

# 평가모드로 전환
model.eval()

# 이미지 transform
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

#특징 벡터 추출할 함수 선언
def get_vector(image_name):
  img = Image.open(image_name)
  t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0)) #이미지 텐서변수에 할당
  my_embedding = torch.zeros(512) #특징 벡터가 들어갈 배열 초기화
  def copy_data(m, i, o): #특징벡터 복사 함수
    my_embedding.copy_(o.data.reshape(o.data.size(1))) 
  h = layer.register_forward_hook(copy_data) #텐서변수 값 추출
  model(t_img) #resnet 모델에 적용
  h.remove() #초기화
  return my_embedding.numpy() #특징벡터를 넘파이배열로 바꿈


#학번 정수값 저장

In [ ]:
from google.colab import files
dbfile = files.upload()

In [ ]:
img_db = cv2.imread('파일명')


img_db_gray = cv2.cvtColor(img_db, cv2.COLOR_BGR2GRAY)
img_db_blur = cv2.GaussianBlur(img_db_gray, (5, 5), 0)

#이미지 내의 경계 찾기
ret_db,img_db_th = cv2.threshold(img_db_blur, 127, 255, cv2.THRESH_BINARY_INV)
contours_db, hierachy_db = cv2.findContours(img_db_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#경계 직사각형 찾기
rects_db = [cv2.boundingRect(each) for each in contours_db]
rects_db = sorted(rects_db)

margin = 8
for rect in rects_db:
  print(rect)
  im1_db = cv2.rectangle(img_db_blur.copy(),(rect[0]-margin,rect[1]-margin),(rect[0]+rect[2]+margin, rect[1]+rect[3]+margin),(0,0,0),3)


In [ ]:
#이전에 처리한 이미지 사용
img_for_class = img_db_blur.copy()

#최종 이미지 파일용 배열
db_img = []
margin_pixel = margin

#숫자영역 추출 및 reshape
for rect in rects_db:
  im = img_for_class[rect[1]-margin_pixel:rect[1]+rect[3]+margin_pixel, rect[0]-margin_pixel:rect[0]+rect[2]+margin_pixel]
  row, col = im.shape[:2]
  #정방형 비율 맞추기
  bordersize = max(row,col)
  diff = min(row,col)
  #이미지의 intensity 평균
  bottom = im[row-2: row, 0:col]
  mean = cv2.mean(bottom)[0]
  #정방형 비율로 보정
  border = cv2.copyMakeBorder(
      im, top =0, bottom =0,
      left = int((bordersize - diff) / 2),
      right = int ((bordersize - diff) / 2),
      borderType = cv2.BORDER_CONSTANT,
      value = [255, 255, 255]
  )
  square = border

  #(28,28)로 축소
  resized_img = cv2.resize(square, dsize = (28, 28), interpolation = cv2.INTER_AREA)
  db_img.append(resized_img)

In [ ]:
res_index = []

for i in range(len(db_img)):
  img = db_img[i]
  cv2_imshow(img)
  img = img.reshape(-1,28,28,1)
  input_data = ((np.array(img) / 255) - 1) * -1
  result = np.argmax(cnn_model.predict(input_data), axis = -1)
  res_index.append(int(result))
  print(result)

res_index #학번 정수값 리스트

In [ ]:
#학번 번호별 이미지 저장 28*28크기
db_img

#학번 벡터값 저장

In [ ]:
from PIL import Image
vector_list1 = [] #이미지가 리스트에 저장되어있다.

for i in range(9):
  a = Image.fromarray(db_img[i])
  a.save('db{0}.jpg'.format(i+1)) 
  vector_list1.append(a)


#new data vector(인식)

In [ ]:
img_new = cv2.imread('파일명')


img_new_gray = cv2.cvtColor(img_new, cv2.COLOR_BGR2GRAY)
img_new_blur = cv2.GaussianBlur(img_new_gray, (5, 5), 0)

#이미지 내의 경계 찾기
ret_new,img_new_th = cv2.threshold(img_new_blur, 127, 255, cv2.THRESH_BINARY_INV)
contours_new, hierachy_new = cv2.findContours(img_new_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#경계 직사각형 찾기
rects_new = [cv2.boundingRect(each) for each in contours_new]
rects_new = sorted(rects_new)

margin = 8
for rect in rects_new:
  print(rect)
  im1_new = cv2.rectangle(img_new_blur.copy(),(rect[0]-margin,rect[1]-margin),(rect[0]+rect[2]+margin, rect[1]+rect[3]+margin),(0,0,0),3)

#이전에 처리한 이미지 사용
img_for_class = img_new_blur.copy()

#최종 이미지 파일용 배열
new_img = []
margin_pixel = margin

#숫자영역 추출 및 reshape
for rect in rects_new:
  im = img_for_class[rect[1]-margin_pixel:rect[1]+rect[3]+margin_pixel, rect[0]-margin_pixel:rect[0]+rect[2]+margin_pixel]
  row, col = im.shape[:2]
  #정방형 비율 맞추기
  bordersize = max(row,col)
  diff = min(row,col)
  #이미지의 intensity 평균
  bottom = im[row-2: row, 0:col]
  mean = cv2.mean(bottom)[0]
  #정방형 비율로 보정
  border = cv2.copyMakeBorder(
      im, top =0, bottom =0,
      left = int((bordersize - diff) / 2),
      right = int ((bordersize - diff) / 2),
      borderType = cv2.BORDER_CONSTANT,
      value = [255, 255, 255]
  )
  square = border

  #(28,28)로 축소
  resized_img = cv2.resize(square, dsize = (28, 28), interpolation = cv2.INTER_AREA)
  new_img.append(resized_img)


In [ ]:
res_index2 = []

for i in range(len(new_img)):
  img2 = new_img[i]
  cv2_imshow(img2)
  img2 = img2.reshape(-1,28,28,1)
  input_data2 = ((np.array(img2) / 255) - 1) * -1
  result2 = np.argmax(cnn_model.predict(input_data2), axis = -1)
  res_index2.append(int(result2))
  print(result2)

res_index2 #학번 정수값 리스트

In [ ]:
vector_list2= [] #이미지가 리스트에 저장되어있다.

for i in range(9):
  b = Image.fromarray(new_img[i])
  b.save('nd{0}.jpg'.format(i+1)) 
  vector_list2.append(b)

#유사도 판별

In [ ]:
pic_one_vector = get_vector('파일1') # 벡터 
pic_two_vector = get_vector('파일2')

In [ ]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
pic_one_vector = torch.tensor(pic_one_vector)
pic_two_vector = torch.tensor(pic_two_vector)
cos_sim = cos(pic_one_vector.unsqueeze(0), pic_two_vector.unsqueeze(0))
print('\nCosine similarity: {0}\n'.format(cos_sim))